In [1]:
#lonstaff price path matrix
mx = [[1, 1, 1, 1,1, 1, 1, 1], 
    [1.09, 1.16, 1.22, 0.93,1.11, 0.76, 0.92, 0.88],
    [1.08, 1.26, 1.07, 0.97,1.56, 0.77, 0.84, 1.22],
     [1.34, 1.54, 1.03, 0.92,1.52, 0.90, 1.01, 1.34]]



In [2]:
import numpy as np
strike = 1.10

In [3]:
#converting the price matrix numpy array
Mx = np.zeros((4,8),dtype=np.float64)

In [4]:
#loading the matrix with lonstaff prices
Mx[0,:] =1
Mx[1,:] = mx[1]
Mx[2,:] = mx[2]
Mx[3,:] = mx[3]

In [5]:
#price matrix
Mx

array([[1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [1.09, 1.16, 1.22, 0.93, 1.11, 0.76, 0.92, 0.88],
       [1.08, 1.26, 1.07, 0.97, 1.56, 0.77, 0.84, 1.22],
       [1.34, 1.54, 1.03, 0.92, 1.52, 0.9 , 1.01, 1.34]])

In [6]:
#payoff of the option
payoff = np.maximum(strike - Mx,np.zeros((4, 8),dtype=np.float64))

In [7]:
payoff

array([[0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 ],
       [0.01, 0.  , 0.  , 0.17, 0.  , 0.34, 0.18, 0.22],
       [0.02, 0.  , 0.03, 0.13, 0.  , 0.33, 0.26, 0.  ],
       [0.  , 0.  , 0.07, 0.18, 0.  , 0.2 , 0.09, 0.  ]])

In [8]:
#value matrix of the payoff
value_matrix = np.zeros_like(payoff)
value_matrix

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [9]:
#displaying the last row of the value matrix, where recursive will begin
value_matrix[-1, :] = payoff[-1, :]
value_matrix

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.07, 0.18, 0.  , 0.2 , 0.09, 0.  ]])

In [10]:
#inputs to the value matrix
time_unit = 1
r=0.06
discount = 0.94176

In [11]:
#looping through the value matrix and compaaring with continuation value
for t in range(2, 0 , -1):
    regression = np.polyfit(Mx[t, :], value_matrix[t + 1, :] * discount, 2)
    continuation_value = np.polyval(regression, Mx[t, :])
    value_matrix[t, :] = np.where(payoff[t, :] > continuation_value,
                                          payoff[t, :],
                                          value_matrix[t + 1, :] * discount)

In [12]:
#final value matrix
value_matrix

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.06208383, 0.17      , 0.        ,
        0.34      , 0.18      , 0.22      ],
       [0.        , 0.        , 0.0659232 , 0.13      , 0.        ,
        0.33      , 0.26      , 0.        ],
       [0.        , 0.        , 0.07      , 0.18      , 0.        ,
        0.2       , 0.09      , 0.        ]])

In [13]:
#visualtion of the value matrix to lool like longstaff example
np.transpose(value_matrix)

array([[0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.06208383, 0.0659232 , 0.07      ],
       [0.        , 0.17      , 0.13      , 0.18      ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.34      , 0.33      , 0.2       ],
       [0.        , 0.18      , 0.26      , 0.09      ],
       [0.        , 0.22      , 0.        , 0.        ]])

In [14]:
#discounting the second last column
value_vector = value_matrix[1,:] * discount

In [15]:
#calcualting the expected price of the option
simulations = 8
price = np.sum(value_vector) / float(simulations)

In [16]:
#price of the option
price

0.11443370880098308

In [17]:
# an oop class for lonsatff example
class longstaffexample(object):
    def __init__(self, strike, simulations,MCprice_matrix,M):
        #price path matrix
        self.strike = strike
        self.simulations = simulations
        self.MCprice_matrix = MCprice_matrix
        self.M = M
        self.discount = 0.94170
    def MCprice_matrix(self):
        return MCprice_matrix
    @property
    def MCpayoff(self):
    #"""Returns the inner-value of American Option"""
        payoff = np.maximum(self.strike - self.MCprice_matrix,
                            np.zeros((self.M + 1, self.simulations),
                            dtype=np.float64))
        return payoff
    
    @property
    def value_vector(self):
        value_matrix = np.zeros_like(self.MCpayoff)
        value_matrix[-1, :] = self.MCpayoff[-1, :]
        for t in range(2, 0 , -1):
            regression = np.polyfit(self.MCprice_matrix[t, :], value_matrix[t + 1, :] * self.discount, 2)
            continuation_value = np.polyval(regression, self.MCprice_matrix[t, :])
            value_matrix[t, :] = np.where(self.MCpayoff[t, :] > continuation_value,
                                          self.MCpayoff[t, :],
                                          value_matrix[t + 1, :] * self.discount)

        return value_matrix[1,:] * self.discount
    
    @property
    def price(self): return np.sum(self.value_vector) / float(self.simulations)
    

In [18]:
#calling out the class
le = longstaffexample(1.10, 8,Mx,3)

In [19]:
# price matrix imethod
le.MCprice_matrix

array([[1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [1.09, 1.16, 1.22, 0.93, 1.11, 0.76, 0.92, 0.88],
       [1.08, 1.26, 1.07, 0.97, 1.56, 0.77, 0.84, 1.22],
       [1.34, 1.54, 1.03, 0.92, 1.52, 0.9 , 1.01, 1.34]])

In [20]:
#payoff matrix method
le.MCpayoff

array([[0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 ],
       [0.01, 0.  , 0.  , 0.17, 0.  , 0.34, 0.18, 0.22],
       [0.02, 0.  , 0.03, 0.13, 0.  , 0.33, 0.26, 0.  ],
       [0.  , 0.  , 0.07, 0.18, 0.  , 0.2 , 0.09, 0.  ]])

In [21]:
#value factor method
le.value_vector

array([0.       , 0.       , 0.0584569, 0.160089 , 0.       , 0.320178 ,
       0.169506 , 0.207174 ])

In [22]:
#price method
le.price

0.11442548700373878